# Agents 

## Response framework

In [2]:
from openai import OpenAI
from agents import Agent, Runner

In [17]:
client = OpenAI()

response = client.responses.create(
    instructions="You are a helpful assistant",
    model="gpt-4o-mini",
    input="What will be the output of my python function total = 0; for i in range(10): total += i**2"
)

print(response.output_text)

The provided Python code initializes a variable `total` to 0 and then iterates over the range from 0 to 9 (inclusive) to calculate the sum of the squares of each integer in that range.

Here’s a breakdown of the code:

```python
total = 0
for i in range(10):
    total += i**2
```

During each iteration of the loop, `i` takes on values from 0 to 9, and `i**2` calculates the square of `i`. The results for each iteration are as follows:

- When `i = 0`: \( 0^2 = 0 \)
- When `i = 1`: \( 1^2 = 1 \)
- When `i = 2`: \( 2^2 = 4 \)
- When `i = 3`: \( 3^2 = 9 \)
- When `i = 4`: \( 4^2 = 16 \)
- When `i = 5`: \( 5^2 = 25 \)
- When `i = 6`: \( 6^2 = 36 \)
- When `i = 7`: \( 7^2 = 49 \)
- When `i = 8`: \( 8^2 = 64 \)
- When `i = 9`: \( 9^2 = 81 \)

Now, we can sum all these values:

\[
0 + 1 + 4 + 9 + 16 + 25 + 36 + 49 + 64 + 81 = 285
\]

Therefore, the final output of `total` after the loop completes will be **285**.


## Agentic framework

Agents - is a new framework, built on top of basic responses/completion API. 

Agentic framework introduces few abstractions to build agentic AI apps more efficient:
* Agent  -  LLMs equipped with instructions and tools
* Handoffs - a way to coordinate and delegate between multiple agents
* Guardrails -  input validations in parallel to agents

In [ ]:
from agents import Agent, Runner
agent = Agent(name="Assistant", instructions="You are a helpful assistant", model="gpt-4.1")

result = await Runner.run(agent, "What will be the output of my python function total = 0; for i in range(10): total += i**2")
print(result.final_output)

Let's analyze your Python code:

```python
total = 0
for i in range(10):
    total += i**2
```

**Step-by-step calculation:**

For `i` in `range(10)`, i.e., `i` from 0 to 9, you add `i**2` to `total`.

- When i = 0: total += 0² = 0   => total = 0
- When i = 1: total += 1² = 1   => total = 1
- When i = 2: total += 2² = 4   => total = 5
- When i = 3: total += 3² = 9   => total = 14
- When i = 4: total += 4² = 16  => total = 30
- When i = 5: total += 5² = 25  => total = 55
- When i = 6: total += 6² = 36  => total = 91
- When i = 7: total += 7² = 49  => total = 140
- When i = 8: total += 8² = 64  => total = 204
- When i = 9: total += 9² = 81  => total = 285

So, **the output of your code will be:**  
```
285
```

If you want to **see this output**, add a print statement:

```python
print(total)
```

**Final code:**
```python
total = 0
for i in range(10):
    total += i**2
print(total)   # Output will be 285
```


The runner then runs a loop:
1. We call the LLM for the current agent, with the current input.
2. The LLM produces its output.
    * If the LLM returns a final_output, the loop ends and we return the result.
    * If the LLM does a handoff, we update the current agent and input, and re-run the loop.
    * If the LLM produces tool calls, we run those tool calls, append the results, and re-run the loop.
3. If we exceed the max_turns passes, we raise an exception.

In [ ]:
#this should fail
result = Runner.run_sync(agent, "What will be the output of my python function total = 0; for i in range(10): total += i**2")
print(result.final_output)

RuntimeError: This event loop is already running

In **Jupyter notebooks**, always use **await Runner.run(...)** instead of run_sync(...). 

Jupyter already runs an event loop, and trying to start another will cause errors.

## Hosted tools

In [58]:
from agents import Agent, Runner, WebSearchTool

In [61]:
search_agent = Agent(
    name="Assistant",
    tools=[
        WebSearchTool(),
    ],
)

In [65]:
result = await Runner.run(agent, "What the weather in London is like today?")
print(result.final_output)

I don’t have real-time data access, so I can’t provide the current weather in London right now. For the latest weather updates, please check a reliable source such as:

- [BBC Weather - London](https://www.bbc.co.uk/weather/2643743)
- [Met Office - London](https://www.metoffice.gov.uk/weather/forecast/gcpvj0v07)
- [Weather.com - London](https://weather.com/weather/today/l/London+United+Kingdom)

If you have enabled a weather app or assistant (like Google Assistant or Siri), you can also ask: “What’s the weather in London today?” for instant results!


In [62]:
result = await Runner.run(search_agent, "What the weather in London is like today?")
print(result.final_output)

As of 1:30 PM on Wednesday, April 16, 2025, in London, the weather is partly sunny with a temperature of 60°F (16°C).

## Weather for London, Greater London, United Kingdom:
Current Conditions: Partly sunny, 60°F (16°C)

Daily Forecast:
* Wednesday, April 16: Low: 42°F (6°C), High: 61°F (16°C), Description: Windy with some clouds, then sunshine
* Thursday, April 17: Low: 46°F (8°C), High: 64°F (18°C), Description: Partly sunny
* Friday, April 18: Low: 49°F (9°C), High: 64°F (18°C), Description: Pleasant with clouds and sunshine
* Saturday, April 19: Low: 45°F (7°C), High: 60°F (16°C), Description: Cloudy
* Sunday, April 20: Low: 47°F (8°C), High: 61°F (16°C), Description: Low clouds
* Monday, April 21: Low: 49°F (9°C), High: 60°F (16°C), Description: Cloudy with a couple of showers in the afternoon
* Tuesday, April 22: Low: 48°F (9°C), High: 63°F (17°C), Description: Partial sunshine with a couple of showers, mainly early in the day


Please note that weather conditions can change, so 

more on buildin tools: https://openai.github.io/openai-agents-python/tools/ 

## Function tools

In [27]:
import json

from typing_extensions import TypedDict, Any
from agents import Agent, FunctionTool, RunContextWrapper, function_tool

In [28]:
class Location(TypedDict):
    lat: float
    long: float

@function_tool  
async def fetch_weather(location: Location) -> str:
    """Fetch the weather for a given location.

    Args:
        location: The location to fetch the weather for.
    """
    # In real life, we'd fetch the weather from a weather API
    return "sunny"

In [29]:
agent_with_a_tool = Agent(
    name="Assistant with tools",
    tools=[fetch_weather],
)

In [30]:
result = await Runner.run(agent_with_a_tool, "What will be the output of my python function total = 0; for i in range(10): total += i**2")
print(result.final_output)

The given Python function will calculate the sum of the squares of the numbers from 0 to 9. Here's the breakdown of what it does:

1. Initializes `total` to 0.
2. Iterates over each integer `i` from 0 to 9.
3. For each `i`, it adds `i**2` (i squared) to `total`.

Here's the process step-by-step:

- When `i = 0`: total += 0\*\*2 = 0
- When `i = 1`: total += 1\*\*2 = 1
- When `i = 2`: total += 2\*\*2 = 4
- When `i = 3`: total += 3\*\*2 = 9
- When `i = 4`: total += 4\*\*2 = 16
- When `i = 5`: total += 5\*\*2 = 25
- When `i = 6`: total += 6\*\*2 = 36
- When `i = 7`: total += 7\*\*2 = 49
- When `i = 8`: total += 8\*\*2 = 64
- When `i = 9`: total += 9\*\*2 = 81

Adding up these values:

0 + 1 + 4 + 9 + 16 + 25 + 36 + 49 + 64 + 81 = 285

So, the output of the function will be `total = 285`.


In [32]:
result = await Runner.run(agent, "What the weather in London is like today?")
print(result.final_output)

I don't have access to real-time data, including current weather updates. For the most accurate and up-to-date weather in London today, please check a reliable weather website like [BBC Weather](https://www.bbc.co.uk/weather), [Weather.com](https://weather.com), or a weather app on your smartphone.

If you tell me what you’re planning to do, I can give you general advice on London’s typical weather for this time of year!


result = await Runner.run(agent_with_a_tool, "What the weather in London is like today?")
print(result.final_output)

## Agent as a tool

In [34]:
from agents import Agent, Runner
import asyncio

In [45]:
russian_agent = Agent(
    name="Russian agent",
    instructions="You translate the user's message to Russian",
)

french_agent = Agent(
    name="French agent",
    instructions="You translate the user's message to French",
)

In [47]:
orchestrator_agent = Agent(
    name="orchestrator_agent",
    instructions=(
        "You are a translation agent. You use the tools given to you to translate."
        "If asked for multiple translations, you call the relevant tools."
    ),
    tools=[
        russian_agent.as_tool(
            tool_name="translate_to_russian",
            tool_description="Translate the user's message to Spanish",
        ),
        french_agent.as_tool(
            tool_name="translate_to_french",
            tool_description="Translate the user's message to French",
        ),
    ],
)

In [51]:
result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in Thai.")
print(result.final_output)

"Hello, how are you?" in Thai is "สวัสดี คุณเป็นอย่างไรบ้าง?"


In [52]:
for step in result.raw_responses:
    print(step)
    print("\n")

ModelResponse(output=[ResponseOutputMessage(id='msg_67ffabc71ac88192ad95843edfba8e27061d3a88aa36cb1d', content=[ResponseOutputText(annotations=[], text='"Hello, how are you?" in Thai is "สวัสดี คุณเป็นอย่างไรบ้าง?"', type='output_text')], role='assistant', status='completed', type='message')], usage=Usage(requests=1, input_tokens=125, output_tokens=24, total_tokens=149), response_id='resp_67ffabc64ea0819294f4745c1ee037fa061d3a88aa36cb1d')




In [53]:
result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in French.")
print(result.final_output)

"Hello, how are you?" in French is: "Bonjour, comment ça va ?"


In [54]:
for step in result.raw_responses:
    print(step)
    print("\n")

ModelResponse(output=[ResponseFunctionToolCall(arguments='{"input":"Hello, how are you?"}', call_id='call_PqabFSFmzKoI5O0LmQbjEp31', name='translate_to_french', type='function_call', id='fc_67ffabe23c408192b703099f1cfdfdbc0d52e9dde6ef70ca', status='completed')], usage=Usage(requests=1, input_tokens=125, output_tokens=22, total_tokens=147), response_id='resp_67ffabe1b1d481929ca2a51866d6629d0d52e9dde6ef70ca')


ModelResponse(output=[ResponseOutputMessage(id='msg_67ffabe42edc8192a768a69207b23e640d52e9dde6ef70ca', content=[ResponseOutputText(annotations=[], text='"Hello, how are you?" in French is: "Bonjour, comment ça va ?"', type='output_text')], role='assistant', status='completed', type='message')], usage=Usage(requests=1, input_tokens=163, output_tokens=20, total_tokens=183), response_id='resp_67ffabe3b43081929cf41dc8757565940d52e9dde6ef70ca')




result = await Runner.run(orchestrator_agent, input="Say 'Hello, how are you?' in French and in Russian.")
print(result.final_output)

In [56]:
for step in result.raw_responses:
    print(step)
    print("\n")

ModelResponse(output=[ResponseFunctionToolCall(arguments='{"input":"Hello, how are you?"}', call_id='call_rZGu4hM3boo8KAfsr67NZJdE', name='translate_to_french', type='function_call', id='fc_67ffabf5f7908192a9e297428ac01c3c0642b864a2d3e916', status='completed'), ResponseFunctionToolCall(arguments='{"input":"Hello, how are you?"}', call_id='call_jNyXrxSpzsSMI2wmVIJCZUWL', name='translate_to_russian', type='function_call', id='fc_67ffabf63294819298cc8184fcc421050642b864a2d3e916', status='completed')], usage=Usage(requests=1, input_tokens=0, output_tokens=0, total_tokens=0), response_id='resp_67ffabf4f2c08192abd9dfe1dc633f4c0642b864a2d3e916')


ModelResponse(output=[ResponseOutputMessage(id='msg_67ffabf856f88192b3d2bf62105813160642b864a2d3e916', content=[ResponseOutputText(annotations=[], text='In French: "Bonjour, comment ça va ?"\n\nIn Russian: "Привет, как ты?"', type='output_text')], role='assistant', status='completed', type='message')], usage=Usage(requests=1, input_tokens=198, outpu

more on function calls: https://openai.github.io/openai-agents-python/tools/ 

## Handoffs

In [ ]:
from agents import Agent

In [67]:
history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
)

In [68]:
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's question",
    handoffs=[history_tutor_agent, math_tutor_agent]
)

In [70]:
result = await Runner.run(triage_agent, "What has happened to Maya civilization?")
print(result.final_output)

The Maya civilization was a Mesoamerican culture that existed in present-day Mexico, Guatemala, Belize, Honduras, and El Salvador. It is renowned for its advanced writing system, art, architecture, and mathematical and astronomical systems. Here’s a brief overview of what happened to the Maya civilization:

### Classic Period (c. 250–900 AD)
- **Flourishing City-States**: During this time, the Maya civilization reached its peak with the development of numerous city-states, each with its own ruler. Major cities included Tikal, Copán, Palenque, and Calakmul.
- **Cultural Achievements**: They made significant advances in mathematics, astronomy, and writing with the development of the Maya script.
- **Architecture and Art**: The Maya built impressive stone structures, pyramids, palaces, and observatories.

### Collapse of the Classic Period
- **Environmental Factors**: Deforestation and drought likely contributed to the decline of many city-states.
- **Political Conflicts**: Increased warf

### A short note on Agent as a tool vs Handoffs 

🔧 **Agent as Tool**
* One agent calls another like a function.
* The calling agent stays in control.
* The tool agent just returns data or output, like a tool or utility.

Analogy: You ask a calculator to compute something and you use the result.

Use Agent as Tool when:
* You need to stay in control of the logic.
* The sub-agent’s output is just data to be used.
* You want deterministic, synchronous behavior.

🤝 **Handoff Between Agents**

What it means:
* One agent passes the control flow to another agent entirely.
* The second agent takes over, continues the conversation or task.
* The first agent steps out and doesn’t return until (maybe) the second agent finishes.

Analogy: You go to a therapits, and they hand you off to a specialist who now handles your care.

Use Handoff when:
* The receiving agent needs to fully take over a task.
* You want modular, autonomous agent behavior.
* The system is meant to be open-ended or conversational.

## Guardrails

In [81]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    TResponseInputItem,
    input_guardrail,
)

In [90]:
class TranslationOutput(BaseModel):
    is_translation: bool
    reasoning: str

guardrail_agent = Agent( 
    name="Guardrail check",
    instructions="Check if the user is asking you to translate something.",
    output_type=TranslationOutput,
)

In [95]:
@input_guardrail
async def translation_guardrail(ctx: RunContextWrapper[None], 
                                agent: Agent, input: str | list[TResponseInputItem]) -> GuardrailFunctionOutput:
    
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output, 
        tripwire_triggered= not result.final_output.is_translation,
    )

In [96]:
translation_agent = Agent(  
    name="Translation angent",
    instructions="You are a translation agent. You help users to translate text to different languages",
    input_guardrails=[translation_guardrail],
)

In [97]:
try:
    result = await Runner.run(translation_agent, "Hello, can you help me to write some python code?")
    print("Guardrail didn't trip - this is unexpected")
    print("\n")
    print(result.final_output)
    
except InputGuardrailTripwireTriggered:
    print("Translation guardrail tripped")

Translation guardrail tripped


In [99]:
try:
    result = await Runner.run(translation_agent, "Hello, how do I say 'No, thank you' in Thai?")
    print("Guardrail didn't trip - this is fine")
    print("\n")
    print(result.final_output)
    
except InputGuardrailTripwireTriggered:
    print("Translation guardrail tripped")

Guardrail didn't trip - this is fine


In Thai, you can say "ไม่เอาครับ ขอบคุณครับ" for males or "ไม่เอาค่ะ ขอบคุณค่ะ" for females.


## Context

An LLM only sees what’s in the conversation history. To give it new data, you can:
* Add it to the Agent instructions (static or dynamic).
* Include it in the input passed to Runner.run(...).
* Provide it through function tools the LLM can call on demand.
* Use retrieval or web search tools to fetch data when needed.

In [12]:
from typing import TypedDict
from agents import Agent, Runner, RunContextWrapper, function_tool

In [13]:

@function_tool
async def summarize_last_user_question(ctx: RunContextWrapper[None]) -> str:
    history = ctx.context.get("chat_history", [])
    last_user_msg = next(
        (msg["content"] for msg in reversed(history) if msg.get("role") == "user"),
        None
    )

    if last_user_msg:
        return f"You previously asked: '{last_user_msg[:100]}...'"
    else:
        return "I couldn't find any previous user messages."


In [14]:
history_tool_agent = Agent(
    name="History-aware Assistant",
    instructions="You help users reflect on their recent questions using chat history.",
    tools=[summarize_last_user_question],
)

In [15]:
chat_history = [
    {"role": "user", "content": "How do I write a SQL query to join two tables?"},
    {"role": "assistant", "content": "Would you like an inner join or outer join?"},
    {"role": "user", "content": "I think I need a left join."}
]

result = await Runner.run(
    history_tool_agent,
    input="Can you summarize what I asked earlier?",
    context={"chat_history": chat_history}
)

print(result.final_output)

You asked about needing a left join. Would you like more information or help with that?


In [16]:
from agents import Runner
from app.agents.interviewee_agent_new import create_interviewee_agent

agent = create_interviewee_agent("friendly", "стрессоустойчивость")

chat_history = [
    {"role": "user", "content": "Расскажи о времени, когда ты был под сильным давлением"},
    {"role": "assistant", "content": "Был проект с коротким сроком, пришлось много координировать"},
    {"role": "user", "content": "Вспомни, что я спрашивал до этого?"}
]

result = await Runner.run(
    agent,
    input="Вспомни, что я спрашивал до этого?",
    context={"chat_history": chat_history}
)

print(result.final_output)

К сожалению, я не могу вспомнить предыдущий вопрос. Может, вы сможете напомнить?


In [12]:
#ok
from app.agents.interviewee_agent_new import create_interviewee_agent
from agents import Runner

agent = create_interviewee_agent(persona="toxic", skill="работа в команде")

result = await Runner.run(agent, "Расскажи о конфликте с коллегами и как ты его решал.")
print(result.final_output)

О, конфликты с коллегами – настоящая забава, не правда ли? Ну, если вам так интересно... Вся эта рутина, в которой я якобы "успешно" разрешаю конфликты, обычно сводится к простой магии: поговорить, выслушать, бла-бла-бла, и, возможно, привлечь третью сторону. Довольно скучно. Если бы я хотел развлечений, я бы выбрал что-то другое.

И вот вопрос к вам: вам правда важны такие истории, или это просто часть "завораживающего" собеседования? Может поговорим о чём-то более интересном?


In [13]:
#ok
agent = create_interviewee_agent(persona="shy", skill="инициативность")
result = await Runner.run(agent, "Приведите пример, когда вы проявили инициативу на предыдущей работе.")
print(result.final_output)

Даже не знаю, ну, вроде... я предлагал что-то улучшить, по-моему, автоматизировать процесс какой-то, чтобы быстрее всё было... Наверное, это помогло чуть-чуть.


In [14]:
from app.agents.evaluation_agent import create_evaluation_agent
from app.agents.prompt_builder import render_star_prompt
from agents import Runner

# Пример ответа
answer = """
На прошлом месте работы я заметил, что некоторые клиенты не получали ответов на заявки в течение суток.
Хотя это была не моя зона ответственности, я предложил в команде создать дежурного по клиентским заявкам.
Я собрал статистику, предложил план, и мы внедрили эту практику. Количество просрочек снизилось на 80%.
"""

prompt = render_star_prompt(answer, skill="инициативность")
agent = create_evaluation_agent(prompt)

result = await Runner.run(agent, "Проанализируй ответ по STAR")
print(result.final_output)



Tool call: ResponseFunctionToolCall(arguments='{"Situation":"На прошлом месте работы я заметил, что некоторые клиенты не получали ответов на заявки в течение суток."}', call_id='call_mRTyBhbNaUr20Vk1bshcNLHO', name='extract_situation', type='function_call', id='fc_68029aca269c81928203ffa9fc63c50800c01ad78b032109', status='completed')
Response from extract_situation: {'Situation': 'На прошлом месте работы я заметил, что некоторые клиенты не получали ответов на заявки в течение суток.'}
Tool call: ResponseFunctionToolCall(arguments='{"Task":"Создать практику дежурства по клиентским заявкам с целью снижения количества просрочек в ответах клиентам."}', call_id='call_NADt3csI4rDFbOCg5tuDFhGA', name='extract_task', type='function_call', id='fc_68029acb6954819280b218e89a4b080200f92e79b7654219', status='completed')
Response from extract_task: {'Task': 'Создать практику дежурства по клиентским заявкам с целью снижения количества просрочек в ответах клиентам.'}
Tool call: ResponseFunctionToolCal

In [9]:
from app.agents.evaluation_agent import create_evaluation_agent, generate_feedback
from app.agents.prompt_builder import render_star_prompt
from agents import Runner

# 1. Ответ кандидата (можно подставить любой свой)
answer = """
В команде произошёл конфликт между аналитиком и разработчиком из-за сроков. 
Я выступил медиатором, собрал обе стороны, мы разобрали ожидания и договорились о более гибком подходе к планированию. 
В результате удалось сохранить атмосферу в команде и не сорвать релиз.
"""

# 2. Компетенция, по которой оцениваем
skill = "работа в команде"

# 3. Формируем system prompt
system_prompt = render_star_prompt(answer, skill)

# 4. Создаём агента
agent = create_evaluation_agent(system_prompt)

# 5. Запускаем STAR-анализ
result = await Runner.run(agent, "Проанализируй ответ по STAR")

# 6. Извлекаем результат
from app.agents.evaluation_agent import STAROutput

# 6. Извлекаем результат
star_output = result.final_output  # ← это уже STAROutput
feedback = generate_feedback(star_output, skill)

# 7. Печатаем всё
print("=== STAR-анализ ===")
print(star_output)

print("\n=== Автоматический фидбек ===")
print(feedback)





Tool call: ResponseFunctionToolCall(arguments='{"Situation":"В команде произошёл конфликт между аналитиком и разработчиком из-за сроков. Я выступил медиатором, собрал обе стороны, мы разобрали ожидания и договорились о более гибком подходе к планированию. В результате удалось сохранить атмосферу в команде и не сорвать релиз."}', call_id='call_e4b49RIitYHMF46Y2BThuEdX', name='extract_situation', type='function_call', id='fc_680297f94eac81929fdfc0be1dd2dbc801cc2df4f4cfe9e6', status='completed')
Response from extract_situation: {'Situation': 'В команде произошёл конфликт между аналитиком и разработчиком из-за сроков. Я выступил медиатором, собрал обе стороны, мы разобрали ожидания и договорились о более гибком подходе к планированию. В результате удалось сохранить атмосферу в команде и не сорвать релиз.'}
Tool call: ResponseFunctionToolCall(arguments='{"Task":"Разрешение конфликта между аналитиком и разработчиком из-за сроков, с целью сохранить атмосферу в команде и не сорвать релиз."}', 

In [ ]:
from app.agents.tools.ask_hr_question import ask_hr_question
print(type(ask_hr_question))
print(hasattr(ask_hr_question, "run"))
print(ask_hr_question.run())

<class 'app.agents.tools.ask_hr_question.CustomTool'>
True
А теперь у меня вопрос к вам как к работодателю: Я слышал, что были сокращения на рынке. Это как-то отразилось на вас?


In [8]:
#!!!
from app.agents.interviewee_agent_new import create_interviewee_agent
from agents import Runner

agent = create_interviewee_agent(persona="toxic", skill="работа в команде")

# HR задаёт вопрос кандидату
result = await Runner.run(agent, "Расскажи о конфликте с коллегами и как ты его решал.")
print(result.final_output)

# Теперь — вопрос кандидата HR'у
question_to_hr = ask_hr_question.run()
print("\n[Вопрос кандидата HR'у]")
print(question_to_hr)

Конфликты с коллегами... ну, кто их не имеет? Обычно стараюсь выслушать обе стороны. Ведь слушать — это уже половина успеха. Настраиваю всех на общий интерес, чтобы работали на одну цель, а не только топили за свои жалобы. Если простыми словами — пытаюсь придумывать решения, а не просто сыпать обвинениями.

Был, например, случай. Спорили из-за задач, и я предложил сесть и распределить всё по полочкам. Выяснили, кто сильнее в чём, и успешно закончили проект. Всё просто как дважды два. Хочешь, расскажу, как это на деле бывает?


AttributeError: 'FunctionTool' object has no attribute 'run'

In [5]:
from app.agents.tools.ask_hr_question import ask_hr_question

print(hasattr(ask_hr_question, "name"))  # True
print(hasattr(ask_hr_question, "run"))   # True
print(ask_hr_question.run())   


True
False


AttributeError: 'FunctionTool' object has no attribute 'run'

In [6]:
from app.agents.interviewee_agent_new import create_interviewee_agent
from agents import Runner

agent = create_interviewee_agent(persona="toxic", skill="работа в команде")
result = await Runner.run(agent, "Расскажи о конфликте с коллегами и как ты его решал.")
print(result.final_output)

О, опять этот шаблонный вопрос. Ну что ж, бывало, коллеги не понимали, чего я хочу, приходилось объяснять им как первоклассникам. В итоге, конечно, мы находили общий язык, но ты ведь понимаешь, насколько это утомительно, да? А про твой опыт расскажешь? У вас на работе много конфликты случаются?


In [6]:
from app.agents.value_agent import create_value_agent
from app.agents.prompt_builder import render_value_prompt
from agents import Runner

answer = """
Я увидел, что коллеги часто путаются в задачах, поэтому предложил ввести Kanban-доску и провёл обучение. 
Теперь команда лучше понимает приоритеты.
"""

value_name = "proactivity"  # ← имя YAML-файла без .yaml

system_prompt = render_value_prompt(value_name, answer)
agent = create_value_agent(system_prompt)

result = await Runner.run(agent, "Проанализируй ответ по корпоративной ценности")
print(result.final_output)


conclusion='Соответствует' explanation='Кандидат проявил проактивность, выявив проблему с путаницей в задачах и предложив решение в виде Kanban-доски. Он также взял инициативу на себя, организовав обучение для команды. Такие действия демонстрируют фокус на улучшениях и инициативность, что соответствует корпоративной ценности проактивности.'


In [15]:
from app.agents.interviewee_agent_new import create_interviewee_agent
from app.agents.evaluation_agent import create_evaluation_agent, generate_feedback
from app.agents.value_agent import create_value_agent
from app.agents.prompt_builder import render_star_prompt, render_value_prompt
from app.agents.tools.ask_hr_question import ask_hr_question
from agents import Runner

# === 1. Настройки интервью ===
persona = "shy"
skill = "стрессоустойчивость"
value_name = "responsibility"
hr_question = "Расскажите о ситуации, когда вы столкнулись с сильным давлением и как вы с ним справились"

# === 2. Кандидат отвечает ===
interviewee = create_interviewee_agent(persona=persona, skill=skill)
candidate_response = await Runner.run(interviewee, hr_question)
print("=== Ответ кандидата ===")
print(candidate_response.final_output)

# === 3. Оценка по STAR ===
star_prompt = render_star_prompt(candidate_response.final_output, skill)
star_agent = create_evaluation_agent(star_prompt)
star_result = await Runner.run(star_agent, "Проанализируй ответ по STAR")

star_output = star_result.final_output  # ← так правильно


print("\n=== STAR-анализ ===")
print(star_output)

feedback = generate_feedback(star_output, skill)
print("\n=== Фидбек по STAR ===")
print(feedback)


# === 4. Оценка по ценности ===
value_prompt = render_value_prompt(value_name, candidate_response.final_output)
value_agent = create_value_agent(value_prompt)
value_result = await Runner.run(value_agent, "Проанализируй ответ по корпоративной ценности")
print("\n=== Оценка по ценности ===")
print(value_result.final_output)

# === 5. Вопрос кандидата к HR ===
print("\n=== Вопрос кандидата HR'у ===")
async def simulate_candidate_question():
    return await ask_hr_question.run_async([])

# И вызывать так:
print(await simulate_candidate_question())


=== Ответ кандидата ===
Ну, даже не знаю... Вроде был случай с проектом... очень много всего нужно было сделать... Хм, я пытался планировать задачи и, наверное, просто старался спокойно работать. Думаю, мы успели вовремя. Было непросто, но как-то справились...
Response from extract_situation: Error generating response: Error code: 400 - {'error': {'message': "Invalid value: 'interviewee'. Supported values are: 'assistant', 'system', 'developer', and 'user'.", 'type': 'invalid_request_error', 'param': 'input[0]', 'code': 'invalid_value'}}
Response from extract_task: Error generating response: Error code: 400 - {'error': {'message': "Invalid value: 'interviewee'. Supported values are: 'assistant', 'system', 'developer', and 'user'.", 'type': 'invalid_request_error', 'param': 'input[0]', 'code': 'invalid_value'}}
Response from extract_action: Error generating response: Error code: 400 - {'error': {'message': "Invalid value: 'interviewee'. Supported values are: 'assistant', 'system', 'deve

AttributeError: 'FunctionTool' object has no attribute 'run_async'

In [11]:
from app.agents.tools.ask_hr_question import ask_hr_question, Message
from agents import Runner, Agent

# Мини-агент только с этим тулом
agent = Agent(
    name="TestAgent",
    instructions="Задай вопрос HR-у",
    tools=[ask_hr_question]
)

# Запуск через Runner
result = await Runner.run(agent, "Задай вопрос HR-у")
print(result.final_output)


Вот отличный вопрос для HR: "Кто принимает решения о повышении?"


In [ ]:
💼 Сценарий: "Скромный кандидат и стрессоустойчивость"
🎭 Персона:
Тип: Скромный кандидат

Навык на проверку: Стрессоустойчивость

Ценность: Ответственность

🚀 Цель:
✅ Получить ответ кандидата на вопрос HR

✅ Провести STAR-анализ

✅ Провести оценку по ценности

✅ Сгенерировать фидбек

✅ Получить встречный вопрос от кандидата HR'у